In [3]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="tqdm")
warnings.filterwarnings("ignore", category=UserWarning, module="dask.dataframe._pyarrow_compat")

In [17]:
from Albert import Einstein 
from Stephen import Hawking 
from transformers import GenerationConfig

import os
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
instance = Hawking()
_ , tokenizer =instance.huggingface_model_load()
model_hawking = instance.load_model(os.getenv("hawking_path"))

def hawking_answer(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda(0)
    model_outputs = model_hawking.generate(input_ids=input_ids, generation_config=GenerationConfig(min_new_tokens= 40, num_beams=1, do_sample=True, top_p=0.5, top_k=0, temperature=0.5,repetition_penalty=2.5))
    model_text_output = tokenizer.decode(model_outputs[0], skip_special_tokens=True)
    return model_text_output

instance_ = Einstein()
_ , tokenizer =instance_.huggingface_model_load()
model_eintein = instance_.load_model(os.getenv("eintein_path"))

def einstein_answer(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda(0)
    model_outputs = model_eintein.generate(input_ids=input_ids, generation_config=GenerationConfig(min_new_tokens= 80, max_new_tokens=200, num_beams=1, do_sample=True, top_p=0.9, top_k=0, temperature=0.5,repetition_penalty=2.5))

    model_text_output = tokenizer.decode(model_outputs[0], skip_special_tokens=True)
    return model_text_output

In [12]:
prompt="Ask question based on answer:\n\nAnswer\n\nA black hole is a region in space where the gravitational force is so strong that even light cannot escape. This region may form as a result of the collapse of a star. Square holes absorb the matter around them, sucking everything, including light, into space. They are called black because even light cannot escape. Because light cannot withstand the gravitational force and is pulled in. Black holes are mysterious and complex phenomena because they are difficult to observe..\n\nQuestion"
prompt

prompt1 = "\n\nAsk question based on answer:\n\nAnswer\n\n{x}\n\nQuestion"
prompt2 = "\n\nAnswer the following question:\n\n{x}\n\nAnswer"

In [16]:
stephen = hawking_answer(prompt)
print("stephen: ", stephen)
print()

albert = einstein_answer(prompt2.format(x=stephen))
print("albert: ", albert)
print()

stephen = hawking_answer(prompt1.format(x=albert))
print("stephen: ", stephen)
print()

albert = einstein_answer(prompt2.format(x=stephen))
print("albert: ", albert)
print()

stephen = hawking_answer(prompt1.format(x=albert))
print("stephen: ", stephen)
print()

albert = einstein_answer(prompt2.format(x=stephen))
print("albert: ", albert)
print()


c:\Users\sbnkr\anaconda3\envs\llm\lib\site-packages\transformers\generation\utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


stephen:  What does the black hole absorb the matter around them?

albert:  The black hole absorbs the matter around them by absorbing it through their own chambers. It is an important part of this process, and one can use these materials to make up for its lack of water or heat energy. In this case, a large area in the vicinity of the Black Hole contains many molecules that are composed of particles that form bonds between the surface of the white dwarf galaxy. These structures have been found to be highly resistant to sunlight but have also become extremely difficult to detect because they cannot support the presence of gravity or other objects. This means that the mass of a small black hole has fallen apart, which causes the darkest areas to collapse in the darkness. However, there are several ways to conserve and preserve the material around them: 1. A dense layer of green algae, such as an alpha-like structure that surrounds the surrounding region, may contain some kind of flaky g

In [1]:
# from transformers import AutoModelForSequenceClassification, AutoTokenizer
# model_name ="sabankara/albert_einstein"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [67]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory

from typing import Any, List, Mapping, Optional

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM

In [2]:
# class CustomLanguageModel:
#     def __init__(self, model, tokenizer):
#         self.model = model
#         self.tokenizer = tokenizer

#     def predict(self, input_text):
#         input_ids = self.tokenizer(input_text, return_tensors="pt").input_ids.cuda(0)
#         model_outputs = self.model.generate(
#             input_ids=input_ids,
#             generation_config=GenerationConfig(
#                 min_new_tokens=80,
#                 max_new_tokens=200,
#                 num_beams=1,
#                 do_sample=True,
#                 top_p=0.9,
#                 top_k=0,
#                 temperature=0.5,
#                 repetition_penalty=2.5
#             )
#         )

#         model_text_output = self.tokenizer.decode(
#             model_outputs[0], skip_special_tokens=True
#         )
#         return model_text_output


# custom_model = CustomLanguageModel(model=model_eintein, tokenizer=tokenizer)

# memory = ConversationBufferWindowMemory(k=1)
# conversation = ConversationChain(
#     llm=custom_model, 
#     memory = memory,
#     verbose=False
# )
# #NOTE k=1 hafızayı sınırlandırdı
# conversation.predict(input="Hi, my name is Andrew")
# # conversation.predict(input="What is 1+1?")
# # conversation.predict(input="What is my name?")